In [1]:
%load_ext autoreload
%autoreload 2
from utils_v2 import *

In [2]:
import seaborn as sns
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

为了验证算法的正确性, 我们取excel中的配对, excel中默认配对是建行和工行

取建行和工行的数据

In [3]:
# 如需验证其他pair结果, 可以同时修改excel的pair公司和下面的代码即可
code1 = '939'
code2 = '1398'

In [9]:
try:
    data = get_data('data/data.zip')
except:
    data = get_data('data/data.csv')

In [10]:
data_pair = data.loc[(slice('2016-01-10', '2018-12-04'), [code1, code2]),:]

注意这里的Adj Close的数据精度只有小数点后3位, 而excel里面不止这个数据精度

In [12]:
data_pair['Adj Close'][0]

3.5710000000000002

In [13]:
data_pair.head()

high   open    low       volume  Adj Close
date       code                                             
2016-01-11 1398  3.596  3.571  3.554  281691602.0      3.571
           939   4.209  4.192  4.158  319178128.0      4.175
2016-01-12 1398  3.622  3.596  3.495  337288458.0      3.520
           939   4.226  4.217  4.141  256018843.0      4.150
2016-01-13 1398  3.622  3.571  3.512  237935728.0      3.528

In [14]:
data_pair.tail()

high  open   low       volume  Adj Close
date       code                                          
2018-11-30 939   6.68  6.62  6.58  274977906.0       6.67
2018-12-03 1398  5.75  5.68  5.66  294301102.0       5.71
           939   6.88  6.85  6.80  426799663.0       6.82
2018-12-04 1398  5.76  5.71  5.69  245585851.0       5.76
           939   6.82  6.80  6.75  331390610.0       6.80

In [18]:
res = single_pair_trading(data_pair, code1, code2, method='all', period=245,
                         rt_period=0, rsi_period=14, upper=70, lower=30)

In [19]:
ratio = res['ratio']

对比excel中的ratio, 观察结果

通过仔细观察, 我们可以看到ratio在最上面的18年的数据, ratio的值和excel中的ratio是完全一样的;

而当我们把数据拉到最下面, ratio的值小数点前3位还能保持一致, 原因就是数据精度的问题

后面我们会看到这个数据精度到底有多少的影响

In [20]:
ratio.sort_index(ascending=False)

date
2018-12-04    1.180556
2018-12-03    1.194396
2018-11-30    1.199640
2018-11-29    1.202899
2018-11-28    1.196780
2018-11-27    1.195298
2018-11-26    1.195652
2018-11-23    1.195572
2018-11-22    1.195572
2018-11-21    1.197053
2018-11-20    1.200000
2018-11-19    1.203670
2018-11-16    1.202578
2018-11-15    1.201479
2018-11-14    1.197761
2018-11-13    1.201493
2018-11-12    1.202247
2018-11-09    1.204120
2018-11-08    1.200364
2018-11-07    1.193784
2018-11-06    1.200000
2018-11-05    1.193015
2018-11-02    1.183453
2018-11-01    1.175047
2018-10-31    1.173585
2018-10-30    1.174242
2018-10-29    1.168561
2018-10-26    1.173004
2018-10-25    1.174242
2018-10-24    1.175803
                ...   
2016-02-24    1.176541
2016-02-23    1.171209
2016-02-22    1.173990
2016-02-19    1.179379
2016-02-18    1.177559
2016-02-17    1.161592
2016-02-16    1.163895
2016-02-15    1.162031
2016-02-12    1.158745
2016-02-11    1.163057
2016-02-05    1.166173
2016-02-04    1.165567
2016-0

对比excel中L2和N2单元格的值

In [21]:
res['long_return']

0.00059422796279257355

In [22]:
res['long_short_return']

-0.00040874532880768948

因为excel中的数据都是做过填充的, 填充的意思是放假的期间也是有数据的, 使用method的是forward fill;

比如, 观察excel表中2016-01-16/17两天是用2016-01-15的数据填充的;

所以为了避免数据和excel中有较大出入, 才进行数据填充; 需要说明的是, 我在实际处理的过程中, 是没有去做数据填充的, 因为休息日本身就是没有数据.

In [23]:
data_pair_fill = data_pair.reset_index().set_index('date').groupby('code')\
.resample('D').first().fillna(method='ffill').drop('code', axis=1).\
reset_index().set_index(['date', 'code'])

In [24]:
res_fill = single_pair_trading(data_pair_fill, '939', '1398', method='all', period=245,
                         rt_period=0, rsi_period=14, upper=70, lower=30)

In [25]:
ratio_fill = res_fill['ratio']

In [26]:
ratio_fill.sort_index(ascending=False)

date
2018-12-04    1.180556
2018-12-03    1.194396
2018-12-02    1.199640
2018-12-01    1.199640
2018-11-30    1.199640
2018-11-29    1.202899
2018-11-28    1.196780
2018-11-27    1.195298
2018-11-26    1.195652
2018-11-25    1.195572
2018-11-24    1.195572
2018-11-23    1.195572
2018-11-22    1.195572
2018-11-21    1.197053
2018-11-20    1.200000
2018-11-19    1.203670
2018-11-18    1.202578
2018-11-17    1.202578
2018-11-16    1.202578
2018-11-15    1.201479
2018-11-14    1.197761
2018-11-13    1.201493
2018-11-12    1.202247
2018-11-11    1.204120
2018-11-10    1.204120
2018-11-09    1.204120
2018-11-08    1.200364
2018-11-07    1.193784
2018-11-06    1.200000
2018-11-05    1.193015
                ...   
2016-02-09    1.166173
2016-02-08    1.166173
2016-02-07    1.166173
2016-02-06    1.166173
2016-02-05    1.166173
2016-02-04    1.165567
2016-02-03    1.165702
2016-02-02    1.174728
2016-02-01    1.180798
2016-01-31    1.177126
2016-01-30    1.177126
2016-01-29    1.177126
2016-0

In [27]:
ratio_fill.describe()

count    1059.000000
mean        1.198451
std         0.038797
min         1.078202
25%         1.178596
50%         1.202091
75%         1.226324
max         1.301661
Name: Adj Close, dtype: float64

填充数据之后的结果和excel中的结果更接近了, 数据差异原因主要就是数据精度的差异, 但是最后的结果的偏差其实并不大

In [28]:
res_fill['long_return']

0.00041422718167199145

In [29]:
res_fill['long_short_return']

-0.00024893872500236753

In [48]:
res_fill.keys()

dict_keys(['ratio', 'real_signal', 'long_asset', 'long_return', 'long_short_return', 'rsi', 'rsi_mask'])

- 我们看一下交易信号
    - 在单边交易中, 1代表long左边的资产, -1代表long右边的资产
    - 在双边交易中, 1代表long左边的资产,同时short右边的资产; -1则是反过来的.
    
需要说明的是, 这里计算出来的信号是做过shift(1)处理的, 所以和excel对比的时候, 要稍微注意一下;

比如18-12-04, 我们这里是-1, 对应到excel中18-12-03那一行的E/F列的0和1;

18-12-03, 我们这里是1, 对应到excel中18-12-02那一行的E/F列的1和0;

In [30]:
res_fill['real_signal'].sort_index(ascending=False)

date
2018-12-04   -1.0
2018-12-03    1.0
2018-12-02    1.0
2018-12-01    1.0
2018-11-30    1.0
2018-11-29   -1.0
2018-11-28   -1.0
2018-11-27   -1.0
2018-11-26   -1.0
2018-11-25   -1.0
2018-11-24   -1.0
2018-11-23   -1.0
2018-11-22   -1.0
2018-11-21    1.0
2018-11-20    1.0
2018-11-19    1.0
2018-11-18    1.0
2018-11-17    1.0
2018-11-16    1.0
2018-11-15   -1.0
2018-11-14    1.0
2018-11-13    1.0
2018-11-12    1.0
2018-11-11    1.0
2018-11-10    1.0
2018-11-09    1.0
2018-11-08   -1.0
2018-11-07    1.0
2018-11-06   -1.0
2018-11-05   -1.0
             ... 
2016-02-09   -1.0
2016-02-08   -1.0
2016-02-07   -1.0
2016-02-06   -1.0
2016-02-05   -1.0
2016-02-04   -1.0
2016-02-03    1.0
2016-02-02    1.0
2016-02-01    1.0
2016-01-31    1.0
2016-01-30    1.0
2016-01-29    1.0
2016-01-28    1.0
2016-01-27   -1.0
2016-01-26   -1.0
2016-01-25   -1.0
2016-01-24   -1.0
2016-01-23   -1.0
2016-01-22   -1.0
2016-01-21   -1.0
2016-01-20    1.0
2016-01-19   -1.0
2016-01-18   -1.0
2016-01-17   -1.0
2016-

In [31]:
asset_1_close = select_code(data_pair_fill, code1)['Adj Close']
asset_2_close = select_code(data_pair_fill, code2)['Adj Close']
ratio = asset_1_close / asset_2_close

position_side_series = position_side(ratio, 'all', 245)
real_signal = position_side_series.shift(1)
long_asset = real_signal.apply(lambda x : code1 if x==1 else
                                      (code2 if x==-1 else np.nan))

asset_1_pct = asset_1_close.pct_change()
asset_2_pct = asset_2_close.pct_change()

# long_return = ((1+real_signal)/2 * asset_1_pct +  (1-real_signal)/2 * asset_2_pct)[-rt_period:].mean()
# long_short_return = (real_signal * (asset_1_pct - asset_2_pct))[-rt_period:].mean()

这个是对应到excel表中的column G

In [32]:
((1+real_signal)/2 * asset_1_pct +  (1-real_signal)/2 * asset_2_pct).\
sort_index(ascending=False)

date
2018-12-04    0.008757
2018-12-03    0.022489
2018-12-02    0.000000
2018-12-01    0.000000
2018-11-30    0.004518
2018-11-29   -0.012522
2018-11-28    0.010850
2018-11-27    0.001812
2018-11-26    0.018450
2018-11-25    0.000000
2018-11-24    0.000000
2018-11-23    0.000000
2018-11-22   -0.001842
2018-11-21    0.003086
2018-11-20   -0.012195
2018-11-19    0.004594
2018-11-18    0.000000
2018-11-17    0.000000
2018-11-16    0.004615
2018-11-15    0.009328
2018-11-14   -0.003106
2018-11-13    0.003115
2018-11-12   -0.001555
2018-11-11    0.000000
2018-11-10    0.000000
2018-11-09   -0.024279
2018-11-08    0.003656
2018-11-07   -0.001529
2018-11-06    0.001838
2018-11-05   -0.021583
                ...   
2016-02-09    0.000000
2016-02-08    0.000000
2016-02-07    0.000000
2016-02-06    0.000000
2016-02-05    0.012597
2016-02-04    0.015535
2016-02-03   -0.015183
2016-02-02   -0.027771
2016-02-01   -0.004235
2016-01-31    0.000000
2016-01-30    0.000000
2016-01-29    0.023980
2016-0

这个是对应到excel表中的column H

In [33]:
(real_signal * (asset_1_pct - asset_2_pct)).sort_index(ascending=False)

date
2018-12-04    0.011689
2018-12-03   -0.004490
2018-12-02    0.000000
2018-12-01    0.000000
2018-11-30   -0.002728
2018-11-29   -0.005049
2018-11-28   -0.001253
2018-11-27    0.000296
2018-11-26   -0.000068
2018-11-25   -0.000000
2018-11-24   -0.000000
2018-11-23   -0.000000
2018-11-22    0.001235
2018-11-21   -0.002469
2018-11-20   -0.003021
2018-11-19    0.000911
2018-11-18    0.000000
2018-11-17    0.000000
2018-11-16    0.000919
2018-11-15   -0.003133
2018-11-14   -0.003106
2018-11-13   -0.000630
2018-11-12   -0.001555
2018-11-11    0.000000
2018-11-10    0.000000
2018-11-09    0.003043
2018-11-08   -0.005532
2018-11-07   -0.005199
2018-11-06   -0.005866
2018-11-05   -0.007905
                ...   
2016-02-09   -0.000000
2016-02-08   -0.000000
2016-02-07   -0.000000
2016-02-06   -0.000000
2016-02-05   -0.000527
2016-02-04    0.000118
2016-02-03   -0.007625
2016-02-02   -0.005023
2016-02-01    0.003096
2016-01-31    0.000000
2016-01-30    0.000000
2016-01-29   -0.004058
2016-0